In [3]:
import time
import uuid
import boto3
import sagemaker

sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [2]:
model_training_s3_uri = 's3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-100GB/'
#model_training_s3_uri = 's3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-100GB-all/' # doesn't pick up the 01-

### Configure the algorithm and training job

The Training Job hyperparameters are set. For more information on XGBoost Hyperparameters, 
see https://xgboost.readthedocs.io/en/latest/parameter.html.

In [4]:
container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

output_content_type = "parquet"

train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=model_training_s3_uri,
    content_type=train_content_type,
    distribution='ShardedByS3Key',
#    input_mode='FastFile'
)

hyperparameters = {
    "eta": "0.2",
    "gamma": "4",
    "max_depth": "5",
    "min_child_weight": "6",
    "num_round": "50",
    "objective": "reg:squarederror",
    "subsample": "0.7"
}

### Start the Training Job

The TrainingJob configurations are set using the SageMaker Python SDK Estimator, and which is fit using 
the training data from the Processing Job that was run earlier.

In [5]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    hyperparameters=hyperparameters,
    instance_count=12,
    instance_type="ml.m5.24xlarge",
    volume_size=200,
#    input_mode='FastFile'
)

In [6]:
import time
import sagemaker

# XX minutes - m5.24xlarge (12 nodes) - 100GB
training_job_results = estimator.fit({"train": train_input})
print(training_job_results)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-03-13-03-46-33-690


2023-03-13 03:46:34 Starting - Starting the training job...
2023-03-13 03:46:49 Starting - Preparing the instances for training.........
2023-03-13 03:48:19 Downloading - Downloading input data...
2023-03-13 03:48:55 Training - Training image download completed. Training in progress....[2023-03-13 03:49:21.504 ip-10-0-174-0.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-03-13 03:49:21.560 ip-10-0-174-0.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-03-13:03:49:21:INFO] Imported framework sagemaker_xgboost_container.training
[2023-03-13:03:49:21:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2023-03-13:03:49:21:INFO] No GPUs detected (normal if no gpus installed)
[2023-03-13:03:49:21:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-03-13:03:49:21:INFO] Determined 0 GPU(s) available on the instance.
[2023-03-13:03:49:21:INFO] files path: /opt/ml/input/data/